In [1]:
import socket
import sys
import threading
import time
from tkinter import *

# ==== Scan Vars ====
ip_s = 1
ip_f = 1024
log = []
ports = []
target = 'localhost'
protocol = 'TCP'

# ==== Vulnerability Database ====
vuln_db = {
    21: "FTP - Vulnerable to anonymous login.",
    22: "SSH - Check for weak or default passwords.",
    23: "Telnet - Insecure protocol; data sent in plaintext.",
    25: "SMTP - Open relay may be possible.",
    53: "DNS - Potential DNS cache poisoning.",
    80: "HTTP - Check for outdated server software.",
    110: "POP3 - Vulnerable to plaintext credential interception.",
    143: "IMAP - Vulnerable to plaintext credential interception.",
    443: "HTTPS - Check for SSL vulnerabilities.",
    445: "SMB - Vulnerable to EternalBlue exploit (MS17-010).",
    3389: "RDP - Vulnerable to brute-force attacks.",
}

# ==== Mitigation Database ====
mitigation_db = {
    21: "Disable anonymous login or use SFTP.",
    22: "Enforce strong passwords and disable root login.",
    23: "Replace with SSH or other secure protocols.",
    25: "Restrict relay to trusted hosts only.",
    53: "Implement DNSSEC and monitor DNS traffic.",
    80: "Regularly update server software.",
    110: "Use encrypted protocols like POP3S.",
    143: "Switch to encrypted protocols like IMAPS.",
    443: "Ensure SSL/TLS certificates are up-to-date.",
    445: "Apply MS17-010 patch and disable SMBv1.",
    3389: "Enable account lockouts and use VPN for RDP access.",
}

# ==== Scanning Functions ====
def scanPort(target, port, protocol='TCP'):
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM if protocol == 'TCP' else socket.SOCK_DGRAM)
        s.settimeout(2)
        if protocol == 'TCP':
            c = s.connect_ex((target, port))
            if c == 0:
                m = f' Port {port} \t[open] ({protocol})'
                log.append(m)
                ports.append(port)
                listbox.insert("end", str(m))
                checkVulnerability(port)
                updateResult()
        else:  # UDP scanning
            try:
                s.sendto(b"", (target, port))
                s.recvfrom(1024)
                m = f' Port {port} \t[open] ({protocol})'
                log.append(m)
                ports.append(port)
                listbox.insert("end", str(m))
                checkVulnerability(port)
                updateResult()
            except socket.error:
                pass
        s.close()
    except OSError:
        print(f'> Too many open sockets. Port {port}')
    except Exception as e:
        print(f'> Error on port {port}: {e}')
        sys.exit()
    sys.exit()

def checkVulnerability(port):
    if port in vuln_db:
        vuln_msg = f" Vulnerability: {vuln_db[port]}"
        mitigation_msg = f" Mitigation: {mitigation_db.get(port, 'No mitigation available.')}"
        log.append(vuln_msg + " " + mitigation_msg)
        listbox.insert("end", f"{vuln_msg} | {mitigation_msg}")

def updateResult():
    rtext = f" [ {len(ports)} / {ip_f} ] ~ {target} ({protocol})"
    L27.configure(text=rtext)

def startScan():
    global ports, log, target, ip_f, protocol
    clearScan()
    log = []
    ports = []
    ip_s = int(L24.get())
    ip_f = int(L25.get())
    protocol = protocol_var.get()
    log.append(f'> Port Scanner ({protocol})')
    log.append('=' * 14 + '\n')
    log.append(' Target:\t' + str(target))

    try:
        target = socket.gethostbyname(str(L22.get()))
        log.append(' IP Adr.:\t' + str(target))
        log.append(f' Ports: \t[ {ip_s} / {ip_f} ]')
        log.append('\n')
        
        while ip_s <= ip_f:
            try:
                scan = threading.Thread(target=scanPort, args=(target, ip_s, protocol))
                scan.setDaemon(True)
                scan.start()
            except:
                time.sleep(0.01)
            ip_s += 1
    except:
        m = f'> Target {L22.get()} not found.'
        log.append(m)
        listbox.insert(0, str(m))

def saveScan():
    global log, target, ports, ip_f
    log[5] = f" Result:\t[ {len(ports)} / {ip_f} ]\n"
    with open(f'portscan-{target}.txt', mode='wt', encoding='utf-8') as myfile:
        myfile.write('\n'.join(log))

def clearScan():
    listbox.delete(0, 'end')

# ==== GUI ====
gui = Tk()
gui.title('Port Scanner')
gui.geometry("400x600+20+20")

# ==== Colors ====
m1c = '#00ee00'
bgc = '#222222'
dbg = '#000000'
fgc = '#111111'

gui.tk_setPalette(background=bgc, foreground=m1c, activeBackground=fgc, activeForeground=bgc, highlightColor=m1c, highlightBackground=m1c)

# ==== Labels ====
L11 = Label(gui, text="Port Scanner", font=("Helvetica", 16, 'underline'))
L11.place(x=16, y=10)

L21 = Label(gui, text="Target: ")
L21.place(x=16, y=90)

L22 = Entry(gui, text="localhost")
L22.place(x=180, y=90)
L22.insert(0, "localhost")

L23 = Label(gui, text="Ports: ")
L23.place(x=16, y=158)

L24 = Entry(gui, text="1")
L24.place(x=180, y=158, width=95)
L24.insert(0, "1")

L25 = Entry(gui, text="1024")
L25.place(x=290, y=158, width=95)
L25.insert(0, "1024")

L26 = Label(gui, text="Results: ")
L26.place(x=16, y=220)
L27 = Label(gui, text="[ ... ]")
L27.place(x=180, y=220)

# ==== Protocol Selection ====
protocol_var = StringVar(value="TCP")
L28 = Label(gui, text="Protocol: ")
L28.place(x=16, y=120)
protocol_menu = OptionMenu(gui, protocol_var, "TCP", "UDP")
protocol_menu.place(x=180, y=120)

# ==== Ports list ====
frame = Frame(gui)
frame.place(x=16, y=275, width=370, height=215)
listbox = Listbox(frame, width=59, height=10)
listbox.place(x=0, y=0)
scrollbar = Scrollbar(frame)
scrollbar.pack(side=RIGHT, fill=Y)
listbox.config(yscrollcommand=scrollbar.set)
scrollbar.config(command=listbox.yview)

# ==== Buttons / Scans ====
B11 = Button(gui, text="Start Scan", command=startScan)
B11.place(x=16, y=500, width=170)
B21 = Button(gui, text="Save Result", command=saveScan)
B21.place(x=210, y=500, width=170)

# ==== Start GUI ====
gui.mainloop()
